# Imports

In [1]:
# Cell 0 - Asegurar ruta al proyecto
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# Cell 1 - Imports
import os
from pipeline.build_area_graph import build_graph_for_area
from utils.config import SAVE_DIR

import folium
from folium.plugins import Draw

In [3]:
# Parámetros de ejemplo
area_geojson = os.path.join(SAVE_DIR, "selection.geojson")  # o ruta a tu GeoJSON de área
draft      = 1.5       # calado máximo del barco (m)
w_max      = 25.0      # velocidad de viento máxima (kn)
start_date = "2025-06-06"
end_date   = "2025-06-06"  # mismo día para T0
paso_lon   = 0.042      # espaciado de la malla en grados (~3 km)

# Selección de una zona

In [8]:
lat, lon, zoom = 40.35, 0.40, 10
m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles='OpenStreetMap')

folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=1.0
).add_to(m)

draw = Draw(
    export=True,
    filename='selection.geojson',
    position='topright',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'marker': False,
        'circlemarker': False,
        'rectangle': True,
    },
    edit_options={'edit': False}
)
draw.add_to(m)

folium.LayerControl().add_to(m)
m


In [4]:
# Cell 2 - Construir grafo para el área
print("Construyendo grafo para el área seleccionada...")
G = build_graph_for_area(
    area_geojson=area_geojson,
    draft=draft,
    w_max=w_max,
    start_date=start_date,
    end_date=end_date,
    paso_lon=paso_lon,   # ahora se llama 'paso_lon'
    max_neighbors=32,
    neighbor_cells=3,
    alpha_time=1.0,
    beta_comfort=0.1,
    output_path=None  # no guarda en disco, solo devuelve el grafo
)

Construyendo grafo para el área seleccionada...


c:\Users\aigut\OneDrive\Documentos\Sailing-route-generator-for-sailboats\graph\nautical_constraints.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([mesh_df, buoy_df], ignore_index=True, sort=False)


KeyboardInterrupt: 

In [ ]:
# Cell 3 - Ver resultados
import networkx as nx
print(f"Grafo construido: {G.number_of_nodes()} nodos, {G.number_of_edges()} aristas")


In [ ]:
# Cell X – Visualizar el grafo en un mapa Folium

import folium
import networkx as nx

# 1) Cargar el grafo desde tu archivo .gpickle
# Sustituye esta ruta por la ubicación real de tu grafo:
gpath = "../data/raw/grafo/weather_routing_graph.gpickle"
G = nx.read_gpickle(gpath)

# 2) Centrar el mapa en el bounding box del grafo
lats = [data['latitude'] for _, data in G.nodes(data=True)]
lons = [data['longitude'] for _, data in G.nodes(data=True)]
center = [(min(lats) + max(lats)) / 2, (min(lons) + max(lons)) / 2]

m = folium.Map(location=center, zoom_start=10, tiles="OpenStreetMap")

# 3) Dibujar aristas del grafo
for u, v, _ in G.edges(data=True):
    m.add_child(folium.PolyLine(
        locations=[
            (G.nodes[u]['latitude'], G.nodes[u]['longitude']),
            (G.nodes[v]['latitude'], G.nodes[v]['longitude'])
        ],
        weight=1, opacity=0.5, color='blue'
    ))

# 4) Dibujar nodos del grafo
for n, data in G.nodes(data=True):
    m.add_child(folium.CircleMarker(
        location=(data['latitude'], data['longitude']),
        radius=2, color='red', fill=True, fill_opacity=0.7
    ))

# Mostrar el mapa interactivo
m
